In [1]:
from math import exp
from math import sqrt
from QuantLib import *

# 0. Parameters
---------------

In [20]:
maturity_date = Date(15, 1, 2016)
spot_price = 0.98
strike_price = 1.
volatility = 0.20
dividend_rate = 0.05
option_type = Option.Call

risk_free_rate = 0.02
day_count = Actual365Fixed()
calendar = NullCalendar()

calculation_date = Date(8, 5, 2015)
Settings.instance().evaluationDate = calculation_date

# 1. European Option
---------------------

## 1.1 Option Construction

In [21]:
payoff = PlainVanillaPayoff(option_type, strike_price)
exercise = EuropeanExercise(maturity_date)
european_option = VanillaOption(payoff, exercise)

In [22]:
spot_handle = QuoteHandle(SimpleQuote(spot_price))
flat_ts = YieldTermStructureHandle(FlatForward(calculation_date, risk_free_rate, day_count))
dividend_yield = YieldTermStructureHandle(FlatForward(calculation_date, dividend_rate, day_count))
flat_vol_ts = BlackVolTermStructureHandle(BlackConstantVol(calculation_date, calendar, volatility, day_count))

bsm_process = BlackScholesMertonProcess(spot_handle, dividend_yield, flat_ts, flat_vol_ts)

## 1.2 Evaluation (using vanilla option engine)

In [23]:
european_option.setPricingEngine(AnalyticEuropeanEngine(bsm_process))
bsm_price = european_option.NPV()
print("BSM european theoreticl price is {0:.4f}".format(bsm_price))

BSM european theoreticl price is 0.0462


## 1.3 Evaluation (using fast path Black Scholes formula)

In [24]:
ttm = day_count.yearFraction(calculation_date, maturity_date)
forward_price = spot_price * exp((risk_free_rate - dividend_rate) * ttm)
std_dev = volatility * sqrt(ttm)
discount = exp(-risk_free_rate * ttm)

In [25]:
bs_price = blackFormula(option_type, strike_price, forward_price, std_dev, discount)
print("Black - Scholes formula theoreticl price is {0:.4f}".format(bs_price))

Black - Scholes formula theoreticl price is 0.0462


# 2. American Option
----------------

## 2.1 Option Construction
------------------

In [26]:
exercise = AmericanExercise(calculation_date, maturity_date)
american_option = VanillaOption(payoff, exercise)

## 2.2 Evaluation (using BaroneAdesi - Whaley method)

In [27]:
american_option.setPricingEngine(BaroneAdesiWhaleyEngine(bsm_process))
bw_price = american_option.NPV()
print("BaroneAdesi - Whaley theoreticl price is {0:.4f}".format(bw_price))

BaroneAdesi - Whaley theoreticl price is 0.0479


## 2.3 Evaluation (using binomial tree method)

In [28]:
steps = 2000

binomial_engine = BinomialVanillaEngine(bsm_process, 'crr', steps)
american_option.setPricingEngine(binomial_engine)
bt_price = american_option.NPV()
print("Binomial Tree theoreticl price is {0:.4f}".format(bt_price))

Binomial Tree theoreticl price is 0.0478
